# Generators

This is a (very) brief description of generators, what they are, how to use them, and how they are useful.

First, we can create a generator using the structure of list comprehension, only using parentheses rather than square brackets.

In [72]:
g = (i for i in range(15) if i%2==1)
print(type(g))

<class 'generator'>


In [73]:
g

<generator object <genexpr> at 0x0000015D2492FC10>

But as you can see, we cannot directly print out the elements that are contained in the generator <code>g</code>.  We saw this same beforehavior with <code>map()</code>, <code>zip()</code>, and <code>filter()</code> because those functions return generators.  We access the elements in generators, again, using the following methods, including iterating through them with a <code>for</code> statement.

In [74]:
for x in g:
    print(x)

1
3
5
7
9
11
13


In [75]:
[*g]

[]

And, as shown in the cell above, you can only get the values out of an iterator once: you cannot use it again.

In [76]:
g = (i for i in range(15) if i%2==1)
[*g]

[1, 3, 5, 7, 9, 11, 13]

In [77]:
g = (i for i in range(15) if i%2==1)
[x for x in g]

[1, 3, 5, 7, 9, 11, 13]

## Why Use Generators?

- You must if you are using a function like <code>map()</code>, <code>zip()</code>, and <code>filter()</code>.
- They take little memory
  - The contents of the iterator are not computed until they are needed (lazy execution) so it doesn't take up memory
- They are fast

## Alternate Method for Creating a Generator

This method is not required for our work in the Optimization course, but it is nice to know if it is needed.  The key feature in creating a generator with a custom function is the use of the keyword <code>yield</code> rather than <code>return</code>.  <code>yield</code> causes the current value(s) to be returned when the generator is used with the <code>next</code> function in the calling program while causing the function to pause.  The current state of the function is retained, and it will resume execution the next time it is called precisely where execution was previously paused.  The generator function can also be used in <code>for</code> loops and list comprehension.

In [78]:
def odd(n):
    for i in range(n):
        if i%2==1:
            yield i

In [84]:
g = odd(5)
print(next(g))
print(next(g))
print(next(g))

1
3


StopIteration: 

In [80]:
g = odd
for d in g(8):
    print(d)

1
3
5
7


In [85]:
[i for i in odd(10)]

[1, 3, 5, 7, 9]

# <code>itertools.combinations()</code>

The <code>itertools.combinations()</code> function generates all combinations of the given set of items having a specified number of elements.  This function can be used to construct all the terms for a constraint to prevent a subtour among the indices in a given subset.

Note the this function returns a generator.

In [3]:
import itertools

x = [0,1,3]
[*itertools.combinations(x,2)]

[(0, 1), (0, 3), (1, 3)]

In [10]:
n = 4
for m in range(2,n+1):
    print(*itertools.combinations(range(n),m))

(0, 1) (0, 2) (0, 3) (1, 2) (1, 3) (2, 3)
(0, 1, 2) (0, 1, 3) (0, 2, 3) (1, 2, 3)
(0, 1, 2, 3)


The <code>itertools.chain()</code> function permits multiple <code>itertools.combinations()</code> generators to be combined into a single generator.

In [14]:
n = 4
combo = []
for m in range(2,n+1):
    combo.append(itertools.combinations(range(n),m))
print(combo)
combo = itertools.chain(*combo)
print([*combo])
#for c in combo:
#    print(c)

[<itertools.combinations object at 0x0000015D2413C4A0>, <itertools.combinations object at 0x0000015D2413C130>, <itertools.combinations object at 0x0000015D2413C770>]
[(0, 1), (0, 2), (0, 3), (1, 2), (1, 3), (2, 3), (0, 1, 2), (0, 1, 3), (0, 2, 3), (1, 2, 3), (0, 1, 2, 3)]


# Subtour Detection and Constraint Construction Example

The data in this example are constructed so that the MST solution without subtour constraints will have a subtour among the locstion indices $\left\lbrace 0,1,3 \right\rbrace$  The example shows how a constraint is constructed to prevent that subtour in subsequent optimization iterations.

In [86]:
import itertools
import gurobipy as gpy
import numpy as np

''' Generate random distance data in the form of an upper triangular matrix '''
n = 4
locs = range(n)
d = np.random.random((n,n))
d = np.triu(d,1)
d = [d[i][j] for i in range(n) for j in range(i+1,n)]
d[0] = 0.00000001
d[2] = 0.00000001
d[4] = 0.00000001
print(d)

''' Construct Gurobi model and optimize '''
m = gpy.Model('mst_eg')
x = [m.addVar(vtype=gpy.GRB.BINARY,name='x_'+str(i)+'_'+str(j)) for i in range(n) for j in range(i+1,n)]
m.update()
m.setObjective(gpy.quicksum(x[i] * d[i] for i in range(len(x))), gpy.GRB.MINIMIZE)  
m.addLConstr(gpy.quicksum(x), gpy.GRB.EQUAL, rhs=n-1)
m.update()
m.optimize()

''' Print solution '''
for v in m.getVars():
    print(f'{v.varName}: {v.x}')

''' Search for subtours '''
''' Consider all subsets of locations for all possible number of subset sizes m '''
for mm in range(2,n):
    ''' Construct all m-location combinations of the location set '''
    subsets = itertools.combinations(locs,mm)
    #print([*subtours])
    ''' construct constraints for all of the combinations '''
    for subset in subsets:
      print(subset, [f'x_{i}_{j}' for i,j in itertools.combinations(subset,2)])
      num_links = sum([m.getVarByName(f'x_{i}_{j}').x for i,j in itertools.combinations(subset,2)])
      print(f'Subset {subset}; Number of links: {num_links}')
      if num_links >= len(subset):
          ''' Construct subtour constraint for this combination '''
          m.addConstr(gpy.quicksum(m.getVarByName(f'x_{i}_{j}') for i,j in itertools.combinations(subset,2)) <= len(subset) - 1)


''' The model needs to be updated after all constraints are added, otherwise 
    the previous solution is deleted '''
m.update()  
m.optimize()

[1e-08, 0.9782668101726484, 1e-08, 0.9417615143340413, 1e-08, 0.9042656155693023]
Gurobi Optimizer version 10.0.3 build v10.0.3rc0 (win64)

CPU model: Intel(R) Xeon(R) CPU E5-1650 v3 @ 3.50GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 6 physical cores, 12 logical processors, using up to 12 threads

Optimize a model with 1 rows, 6 columns and 6 nonzeros
Model fingerprint: 0x5c8004fe
Variable types: 0 continuous, 6 integer (6 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e-08, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [3e+00, 3e+00]
Found heuristic solution: objective 0.9782668
Presolve removed 1 rows and 6 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 12 available processors)

Solution count 2: 3e-08 0.978267 

Optimal solution found (tolerance 1.00e-04)
Best objective 3.000000000000e-08, best bound 3.0000